In [1]:
from funcs.WLANFuncs import *
from funcs.HelperFuncs import *

from multiprocessing import Pool
import os
import pandas as pd
from glob import glob

%matplotlib inline

pd.options.display.float_format = '{:6f}'.format

import warnings
warnings.filterwarnings("ignore")


# Convert JSON to H5 (good for ML and similar to csv)
## Also adding derivate features
* Is upstream/downstream

In [3]:
procs = 4

ssid = "67Marlie"
pwd = "sayplease"

dirs = glob("/Volumes/tim_details/tim_honours/CAPTURES/*/*")

for dir in dirs:
    # Setting up dirs
    clean_dir_junk(os.path.join(dir, "wlan"))
    make_dir(os.path.join(dir, "wlan_json"))
    make_dir(os.path.join(dir, "wlan_h5"))
    # Running multiprocessing: wlan pcap to h5
    starargs = [(dir, get_name(i)) for i in os.listdir(os.path.join(dir, "wlan"))]
    with Pool(procs, initializer=wlan_to_df_init_mp, initargs=(ssid, pwd)) as pool:
        pool.starmap(wlan_to_df_mp, starargs)
    # Cleaning dirs (removing json dir)
    remove_dir(os.path.join(dir, "wlan_json"))

/Volumes/tim_details/tim_honours/CAPTURES/client_pc_50/v=A3gUpodXMv0 - cap_15
/Volumes/tim_details/tim_honours/CAPTURES/client_pc_50/v=A3gUpodXMv0 - cap_97
/Volumes/tim_details/tim_honours/CAPTURES/client_pc_50/v=A3gUpodXMv0 - cap_2
/Volumes/tim_details/tim_honours/CAPTURES/client_pc_50/v=A3gUpodXMv0 - cap_19
/Volumes/tim_details/tim_honours/CAPTURES/client_pc_50/v=A3gUpodXMv0 - cap_26
/Volumes/tim_details/tim_honours/CAPTURES/client_pc_50/v=A3gUpodXMv0 - cap_49
/Volumes/tim_details/tim_honours/CAPTURES/client_pc_50/v=A3gUpodXMv0 - cap_59
/Volumes/tim_details/tim_honours/CAPTURES/client_pc_50/v=A3gUpodXMv0 - cap_61
/Volumes/tim_details/tim_honours/CAPTURES/client_pc_50/v=A3gUpodXMv0 - cap_7
/Volumes/tim_details/tim_honours/CAPTURES/client_pc_50/v=A3gUpodXMv0 - cap_63
/Volumes/tim_details/tim_honours/CAPTURES/client_pc_50/v=A3gUpodXMv0 - cap_86
/Volumes/tim_details/tim_honours/CAPTURES/client_pc_50/v=A3gUpodXMv0 - cap_84
/Volumes/tim_details/tim_honours/CAPTURES/client_pc_50/v=A3gUpodXM

In [7]:
pd.read_hdf("/Volumes/tim_details/tim_honours/CAPTURES/client_pc_50/v=A3gUpodXMv0/wlan_h5/cap_6.h5", key=H5_WLAN_KEY, mode="r").dtypes

frame.time_relative      float64
frame.number               int64
frame.len                  int32
wlan_radio.signal_dbm       int8
wlan.fc.version             int8
wlan.fc.type                int8
wlan.fc.subtype             int8
wlan.frag                   int8
wlan.seq                   int16
wlan.da                   object
wlan.sa                   object
ip.proto                    int8
ip.src                    object
ip.dst                    object
ip.len                     int16
srcport                    int16
dstport                    int16
is_upstream                 bool
dtype: object

## Investigating streams

## Visualising cumulative bytes by time (binned in time intervals)

In [ ]:
# dirs = glob("/Users/timothylee/Desktop/Uni/Yr5/Honours/honours_dev/2_analyse/wlan_captures")
dirs = glob("/Volumes/tim_details/tim_honours/CAPTURES/*/*")

for dir in dirs:
    # Getting directory and file paths
    h5_dir = os.path.join(dir, "wlan_h5")
    plot_dir = os.path.join(dir, "wlan_plots")
    plot_fp = os.path.join(plot_dir, "all_cumsum.png")
    # Preparing directories
    make_dir(plot_dir)

    # If the plot already exists, then skip
    if os.path.isfile(plot_fp):
        continue

    # Making plots
    fig, ax = WLANFuncs.wlan_cumsum_plot(h5_dir)
    fig.savefig(plot_fp)
    fig.clf();
plt.close();